# Negentropic Coupling Framework - Interactive Demo

This notebook demonstrates the **Negentropic Coupling Framework (NCF)** simulation.

## Overview

NCF models how distributed systems self-stabilize through:
- **Entropy (H)**: Information uncertainty
- **Negentropy (N)**: Order measure (N = 1 - H/Hₘₐₓ)
- **Coherence (C)**: Bidirectional alignment
- **Entropy Velocity (v)**: Rate of change

---

## 1. Setup and Imports

In [ ]:
import sys
import json
import numpy as np
import matplotlib.pyplot as plt

# Import NCF simulation module
sys.path.append('../models')
from NCF_simulation import NCFSimulation, run_simulation, plot_negentropy_evolution

print("NCF Simulation Module loaded successfully!")

## 2. Load Example Data

In [ ]:
# Load example mesh configuration
with open('entropy_mesh_example.json', 'r') as f:
    example_data = json.load(f)

print("Example mesh configuration:")
print(f"  Nodes: {example_data['mesh']['nodes']}")
print(f"  Edges: {len(example_data['mesh']['edges'])}")
print(f"  Mode: {example_data['simulation_parameters']['mode']}")
print(f"  Steps: {example_data['simulation_parameters']['time_steps']}")

## 3. Initialize Simulation

In [ ]:
# Create simulation instance
n_nodes = example_data['mesh']['nodes']
n_edges = len(example_data['mesh']['edges'])

sim = NCFSimulation(n_nodes=n_nodes, n_edges=n_edges)

print(f"Initialized mesh with {n_nodes} nodes and {len(sim.edges)} edges")
print(f"\nSample edges: {sim.edges[:3]}")

## 4. Compute Initial Metrics

In [ ]:
# Compute metrics for first edge
edge = sim.edges[0]

entropy = sim.entropy_field(edge)
negentropy = sim.negentropic_index(edge)
coherence = sim.coherence(edge)
policy = sim.policy(edge)

print(f"Metrics for edge {edge}:")
print(f"  Shannon Entropy (H): {entropy:.4f}")
print(f"  Negentropic Index (N): {negentropy:.4f}")
print(f"  Coherence (C): {coherence:.4f}")
print(f"  Policy: {policy}")

## 5. Run Simulation

In [ ]:
# Run simulation for specified steps
steps = example_data['simulation_parameters']['time_steps']
mode = example_data['simulation_parameters']['mode']

results = []
print(f"Running {steps} simulation steps...\n")

for step in range(steps):
    metrics = sim.evolve()
    results.append(metrics)
    
    if step % 2 == 0:  # Print every other step
        print(f"Step {step}: N={metrics['negentropy']:.3f}, "
              f"C={metrics['coherence']:.3f}, v={metrics['velocity']:.4f}")

print(f"\nSimulation complete!")

## 6. Visualize Results

In [ ]:
# Extract time series data
history = sim.get_history()
times = [h['time'] for h in history]
negentropies = [h['negentropy'] for h in history]
coherences = [h['coherence'] for h in history]
velocities = [h['velocity'] for h in history]

# Create multi-panel plot
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Plot 1: Negentropy
axes[0].plot(times, negentropies, 'b-', linewidth=2, marker='o')
axes[0].set_ylabel('Negentropy (N)', fontsize=12)
axes[0].set_title('Negentropic Evolution', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0.8, color='g', linestyle='--', alpha=0.5, label='Macro threshold')
axes[0].axhline(y=0.3, color='r', linestyle='--', alpha=0.5, label='Defensive threshold')
axes[0].legend()

# Plot 2: Coherence
axes[1].plot(times, coherences, 'g-', linewidth=2, marker='s')
axes[1].set_ylabel('Coherence (C)', fontsize=12)
axes[1].set_title('Coherence Dynamics', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Plot 3: Entropy Velocity
axes[2].plot(times, velocities, 'r-', linewidth=2, marker='^')
axes[2].set_xlabel('Time Step', fontsize=12)
axes[2].set_ylabel('Entropy Velocity (v)', fontsize=12)
axes[2].set_title('Rate of Informational Change', fontsize=14, fontweight='bold')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='k', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nVisualization complete!")

## 7. Summary Statistics

In [ ]:
# Compute summary statistics
final_n = negentropies[-1]
final_c = coherences[-1]
avg_n = np.mean(negentropies)
avg_c = np.mean(coherences)
max_v = max(abs(v) for v in velocities)

print("=" * 50)
print("SIMULATION SUMMARY")
print("=" * 50)
print(f"\nFinal State (t={steps-1}):")
print(f"  Negentropy: {final_n:.4f}")
print(f"  Coherence:  {final_c:.4f}")
print(f"\nAverages:")
print(f"  Mean Negentropy: {avg_n:.4f}")
print(f"  Mean Coherence:  {avg_c:.4f}")
print(f"  Max |Velocity|:  {max_v:.4f}")
print(f"\nSystem Regime:")

if final_n > 0.8 and final_c > 0.8:
    print("  → ORDERED-COHERENT (High throughput)")
elif final_n < 0.3 and final_c < 0.5:
    print("  → CHAOTIC (Defensive mode)")
else:
    print("  → TRANSITIONAL (Adaptive adjustment)")
print("\n" + "=" * 50)

## 8. Explore Edge-Level Details

In [ ]:
# Analyze individual edges
print("Edge-Level Policies (final state):\n")

policy_counts = {"macro": 0, "balanced": 0, "defensive": 0}

for edge in sim.edges[:5]:  # Show first 5 edges
    n = sim.negentropic_index(edge)
    c = sim.coherence(edge)
    p = sim.policy(edge)
    policy_counts[p] += 1
    
    print(f"  Edge {edge}: N={n:.3f}, C={c:.3f} → Policy: {p}")

print(f"\nPolicy Distribution (all {len(sim.edges)} edges):")
for policy, count in policy_counts.items():
    print(f"  {policy}: {count}")

## 9. Theoretical Interpretation

### Key Insights:

1. **Negentropy (N)** measures order:
   - N → 1: Highly ordered, predictable
   - N → 0: Maximum entropy, chaotic

2. **Coherence (C)** measures alignment:
   - High C: Bidirectional agreement
   - Low C: Asymmetric information flow

3. **Entropy Velocity (v)** tracks change:
   - v > 0: Increasing uncertainty
   - v < 0: Decreasing uncertainty (ordering)

4. **Policy Mapping**:
   - `macro`: High-throughput, zero-copy frames (N > 0.8)
   - `balanced`: Moderate throughput (0.3 ≤ N ≤ 0.8)
   - `defensive`: Conservative, ACK-based (N < 0.3)

### Mathematical Foundation:

The system evolves according to:

```
x(t+1) = F(x(t)) + G(N(t), C(t), V(t)) x(t)
```

where **G** is the adaptive coupling operator that adjusts inter-node influence based on measured information fields.

---

**Citation:**
> gsknnft (2025). The Negentropic Coupling Framework (NCF) v1.0.  
> SigilNet Research Series.  
> https://github.com/gsknnft/NegatropicCouplingTheory